<a href="https://colab.research.google.com/github/RifaldiAchmad/Data-Analysis-with-Python/blob/main/Exploratory_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some questions that arise are as follows:
- How has the company's sales and revenue performed in the last few months?
- What products have sold the most and the least?
- What are the demographics of our customers?
- When was the last time a customer made a transaction?
- How often has a customer made a purchase in the last few months?
- How much money has a customer spent in the last few months?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
customers_df = pd.read_csv("https://raw.githubusercontent.com/RifaldiAchmad/Data-Analysis-with-Python/refs/heads/main/Dataset%20Clean/customers_clean.csv")
orders_df = pd.read_csv("https://raw.githubusercontent.com/RifaldiAchmad/Data-Analysis-with-Python/refs/heads/main/Dataset%20Clean/orders_clean.csv")
product_df = pd.read_csv("https://raw.githubusercontent.com/RifaldiAchmad/Data-Analysis-with-Python/refs/heads/main/Dataset%20Clean/products_clean.csv")
sales_df = pd.read_csv("https://raw.githubusercontent.com/RifaldiAchmad/Data-Analysis-with-Python/refs/heads/main/Dataset%20Clean/sales_clean.csv")

# **Explore Customers Data**

In [3]:
customers_df.describe(include="all")

,customer_id,customer_name,gender,age,home_address,zip_code,city,state,country
count,1001.000000,1001,1001,1001.000000,1001,1001.000000,1001,1001,1001
unique,NaN,1000,3,NaN,1000,NaN,961,8,1
top,NaN,fulan 943,Prefer not to say,NaN,3117 Heller PlaceSuite 149,NaN,New Ava,South Australia,Australia
freq,NaN,2,743,NaN,2,NaN,3,140,1001
mean,500.942058,NaN,NaN,49.874126,NaN,5000.693307,NaN,NaN,NaN
std,289.013599,NaN,NaN,17.644663,NaN,2886.084454,NaN,NaN,NaN
min,1.000000,NaN,NaN,20.000000,NaN,2.000000,NaN,NaN,NaN
25%,251.000000,NaN,NaN,34.000000,NaN,2398.000000,NaN,NaN,NaN
50%,501.000000,NaN,NaN,50.000000,NaN,5079.000000,NaN,NaN,NaN
75%,751.000000,NaN,NaN,65.000000,NaN,7454.000000,NaN,NaN,NaN


We have a total of 1001 customers, with ages ranging from 20 to 80 years old. The average age is 49.87 years, with a standard deviation of 17.64 years. This information provides an overview that our customer segmentation is quite broad, spanning from young adults to seniors.

In [4]:
customers_df.groupby("gender").agg({
    "customer_id": "nunique",
    "age": ["max", "min", "mean", "std"]
})

customer_id age                          
                      nunique max min       mean        std
gender                                                     
Female                    115  79  20  49.147826  16.646607
Male                      143  80  20  51.230769  18.462635
Prefer not to say         742  80  20  49.725437  17.644283

Based on the pivot table above, it can be seen that the customers we have are dominated by gender who prefer not to say. On the other hand, their age distribution is quite similar, ranging from 20 to 80 years.

In [5]:
customers_df.groupby("city")["customer_id"].nunique().sort_values(ascending=False)

,customer_id
city,
East Aidan,3
East Sophia,3
New Ava,3
West Jackfort,2
Corkeryshire,2
...,...
Jordanside,1
Josephland,1
Josephmouth,1


In [6]:
customers_df.groupby("state")["customer_id"].nunique().sort_values(ascending=False)

,customer_id
state,
South Australia,139
Queensland,134
New South Wales,132
Northern Territory,125
Western Australia,124
Australian Capital Territory,121
Victoria,121
Tasmania,104


Based on the results above, it can be seen that our customer distribution is fairly even across each city and state. Most of our customers are located in the cities of East Aidan, East Sophia, and New Ava, with three customers each. Additionally, the majority of our customers come from the state of South Australia.

# **Explore Orders Data**

In [16]:
datetime_columns = ["order_date", "delivery_date"]

for column in datetime_columns:
  orders_df[column] = pd.to_datetime(orders_df[column])

In [17]:
delivery_time = orders_df["delivery_date"] - orders_df["order_date"]
delivery_time = delivery_time.apply(lambda x: x.total_seconds())
orders_df["delivery_time (per day)"] = round(delivery_time/86400)
orders_df.head()

,order_id,customer_id,payment,order_date,delivery_date,delivery_time (per day)
0,1,64,30811,2021-08-30,2021-09-24,25.0
1,2,473,50490,2021-02-03,2021-02-13,10.0
2,3,774,46763,2021-10-08,2021-11-03,26.0
3,4,433,39782,2021-05-06,2021-05-19,13.0
4,5,441,14719,2021-03-23,2021-03-24,1.0


We will see about how long the delivery time is

In [20]:
orders_df.describe(include="all")

,order_id,customer_id,payment,order_date,delivery_date,delivery_time (per day)
count,1000.000000,1000.000000,1000.000000,1000,1000,1000.000000
mean,500.500000,506.640000,33972.936000,2021-05-27 18:38:52.800000,2021-06-10 20:31:12,14.078000
min,1.000000,1.000000,10043.000000,2021-01-01 00:00:00,2021-01-03 00:00:00,1.000000
25%,250.750000,275.250000,21329.250000,2021-03-13 18:00:00,2021-03-28 00:00:00,8.000000
50%,500.500000,515.000000,33697.500000,2021-05-27 12:00:00,2021-06-11 12:00:00,14.000000
75%,750.250000,737.250000,46249.000000,2021-08-12 06:00:00,2021-08-24 06:00:00,21.000000
max,1000.000000,1000.000000,59910.000000,2021-10-24 00:00:00,2021-11-20 00:00:00,27.000000
std,288.819436,277.115502,14451.609047,NaN,NaN,7.707225


Based on these results, it can be observed that the average delivery time is 14 days, with a maximum value of 27 days and a minimum value of 1 day.

In [22]:
customer_id_in_orders_df = orders_df["customer_id"].tolist()
customers_df["status"] = customers_df["customer_id"].apply(lambda x: "Active" if x in customer_id_in_orders_df else "Non Active")
customers_df.sample(5)

,customer_id,customer_name,gender,age,home_address,zip_code,city,state,country,status
310,311,fulan 311,Prefer not to say,25,316 Violet LaneApt. 212,2929,Fayburgh,Northern Territory,Australia,Active
631,632,fulan 632,Prefer not to say,55,704 Wright CourtApt. 223,3450,South Anthonyburgh,Northern Territory,Australia,Active
694,695,fulan 695,Prefer not to say,48,65 Stokes DriveSuite 624,6533,O'reillymouth,Western Australia,Australia,Non Active
155,156,fulan 156,Female,24,012 Eli RidgeSuite 727,7891,Kaiberg,Western Australia,Australia,Active
306,307,fulan 307,Female,50,227 Donnelly TerraceSuite 915,4618,Lake Jackmouth,Victoria,Australia,Non Active


In [24]:
customers_df.groupby("status").customer_id.count()

,customer_id
status,
Active,617
Non Active,384


As shown in the pivot table above, there are quite a few customers who have never made a transaction before. This is certainly bad news for us, as nearly 30% of our customers have never placed an order.

# **Explore Orders and Customers Data**

In [37]:
orders_customers_df = pd.merge(
    left=orders_df,
    right=customers_df,
    how="left",
    left_on="customer_id",
    right_on="customer_id"
)
orders_customers_df.head()

,order_id,customer_id,payment,order_date,delivery_date,delivery_time (per day),customer_name,gender,age,home_address,zip_code,city,state,country,status
0,1,64,30811,2021-08-30,2021-09-24,25.0,fulan 64,Prefer not to say,75,4927 Alice MeadowApt. 960,7787,Sanfordborough,South Australia,Australia,Active
1,2,473,50490,2021-02-03,2021-02-13,10.0,fulan 473,Male,61,531 Schmitt BoulevardApt. 010,1744,Annaton,South Australia,Australia,Active
2,3,774,46763,2021-10-08,2021-11-03,26.0,fulan 774,Prefer not to say,34,2096 Wilson MewsApt. 714,8590,West Jonathanshire,Tasmania,Australia,Active
3,4,433,39782,2021-05-06,2021-05-19,13.0,fulan 433,Prefer not to say,46,5777 Mayer PassApt. 881,9728,West Michaelport,Tasmania,Australia,Active
4,5,441,14719,2021-03-23,2021-03-24,1.0,fulan 441,Prefer not to say,53,33 Richards JunctionApt. 478,7650,South Rileyview,Western Australia,Australia,Active


To gather more insights from the two datasets, we will combine them using a join or merge process.

In [35]:
orders_customers_df.groupby("city")["order_id"].nunique().sort_values(ascending=False).reset_index().head(10)

,city,order_id
0,Jordanside,6
1,New Ava,6
2,Lake Rose,5
3,O'keefeton,5
4,East Max,5
5,Port Hannahburgh,5
6,Rubyfort,5
7,West Kai,5
8,Andrewborough,4
9,Port Aaronton,4


Based on the results, it is known that Jordanside and New Ava are the two cities with the highest number of orders.

In [39]:
orders_customers_df.groupby("state")["order_id"].nunique().sort_values(ascending=False)

,order_id
state,
South Australia,148
Queensland,139
Western Australia,130
New South Wales,129
Australian Capital Territory,118
Victoria,118
Tasmania,112
Northern Territory,106


From the pivot table, it is evident that South Australia is the state with the highest number of orders.

In [38]:
orders_customers_df.groupby("gender")["order_id"].nunique().sort_values(ascending=False)

,order_id
gender,
Prefer not to say,725
Female,139
Male,136


Based on the results, it can be seen that most orders are placed by customers who identify as "prefer not to say" for gender. This is consistent with the fact that the majority of our customers belong to this gender group.

In [40]:
orders_customers_df["age_group"] = orders_customers_df["age"].apply(lambda x: "Youth" if x <= 24 else ("Seniors" if x > 64 else "Adults"))
orders_customers_df.groupby("age_group")["order_id"].nunique().sort_values(ascending=False)

,order_id
age_group,
Adults,681
Seniors,226
Youth,93


Based on the results, it is clear that the customers who place the most orders come from the Adults age group.

# **Explore Product and Sales Data**

In [42]:
product_df.describe(include="all")

,product_id,product_type,product_name,size,colour,price,quantity,description
count,1260.000000,1260,1260,1260,1260,1260.000000,1260.000000,1260
unique,NaN,3,35,8,7,NaN,NaN,1260
top,NaN,Shirt,Denim,XS,red,NaN,NaN,"A red coloured, XS sized, Oxford Cloth Shirt"
freq,NaN,420,70,252,180,NaN,NaN,1
mean,629.500000,NaN,NaN,NaN,NaN,105.805556,60.150000,NaN
std,363.874979,NaN,NaN,NaN,NaN,9.704423,11.670573,NaN
min,0.000000,NaN,NaN,NaN,NaN,90.000000,40.000000,NaN
25%,314.750000,NaN,NaN,NaN,NaN,95.750000,50.000000,NaN
50%,629.500000,NaN,NaN,NaN,NaN,108.500000,60.000000,NaN
75%,944.250000,NaN,NaN,NaN,NaN,114.000000,70.000000,NaN


In [43]:
sales_df.describe(include="all")

,sales_id,order_id,product_id,price_per_unit,quantity,total_price
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,2499.500000,503.038200,634.053200,103.501600,1.99240,206.360000
std,1443.520003,285.964418,363.255794,9.195004,0.80751,86.357457
min,0.000000,1.000000,1.000000,90.000000,1.00000,90.000000
25%,1249.750000,258.000000,323.000000,95.000000,1.00000,112.000000
50%,2499.500000,504.500000,635.000000,102.000000,2.00000,204.000000
75%,3749.250000,749.000000,951.000000,112.000000,3.00000,285.000000
max,4999.000000,999.000000,1259.000000,119.000000,3.00000,357.000000


Based on the results, it can be observed that the prices of the items sold range from 90 to 119 dollars. Additionally, we also gathered another interesting piece of information: on average, each customer purchases three items of a single product type per transaction, with a total price of 357 dollars.

In [44]:
product_df.sort_values("price", ascending=False)

,product_id,product_type,product_name,size,colour,price,quantity,description
698,698,Jacket,Parka,L,violet,119,53,"A violet coloured, L sized, Parka Jacket"
665,665,Jacket,Parka,XS,red,119,65,"A red coloured, XS sized, Parka Jacket"
692,692,Jacket,Parka,M,indigo,119,66,"A indigo coloured, M sized, Parka Jacket"
693,693,Jacket,Parka,L,indigo,119,44,"A indigo coloured, L sized, Parka Jacket"
694,694,Jacket,Parka,XL,indigo,119,78,"A indigo coloured, XL sized, Parka Jacket"
...,...,...,...,...,...,...,...,...
1219,1219,Trousers,Pleated,XL,indigo,90,45,"A indigo coloured, XL sized, Pleated Trousers"
1220,1220,Trousers,Pleated,XS,violet,90,58,"A violet coloured, XS sized, Pleated Trousers"
1221,1221,Trousers,Pleated,S,violet,90,50,"A violet coloured, S sized, Pleated Trousers"
1222,1222,Trousers,Pleated,M,violet,90,45,"A violet coloured, M sized, Pleated Trousers"


The most expensive product is a jacket called the Parka, while the cheapest one is named the Bomber.

In [48]:
product_df.groupby("product_type").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price":  ["min", "max"]
})

product_id quantity price     
                nunique      sum   min  max
product_type                               
Jacket              420    25387    90  119
Shirt               420    25263    92  119
Trousers            420    25139    90  119

In [50]:
product_df.groupby("product_name").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price": ["min", "max"]
})

product_id quantity price     
                     nunique      sum   min  max
product_name                                    
Bomber                    35     2083    90   90
Camp Collared             35     2071   112  112
Cardigan                  35     2032   118  118
Cargo Pants               35     2146   106  106
Casual Slim Fit           35     2086   113  113
Chambray                  35     2020   105  105
Chinos                    35     2101   100  100
Coach                     35     2158   115  115
Cords                     35     2260   113  113
Cropped                   35     2085    99   99
Cuban Collar              35     2017    93   93
Denim                     70     4204    92  115
Drawstring                35     2042   104  104
Dress                     35     2125   119  119
Flannel                   35     2135    96   96
Henley                    35     2051    92   92
High-Waisted              35     2198    98   98
Joggers                   35     2107    94   94
Leather                   35     2276   113  113
Linen                     35     2138   116  116
Mandarin Collar           35     2203   108  108
Oxford Cloth              35     2071   114  114
Parka                     35     2201   119  119
Peacoat                   35     2067   102  102
Pleated                   35     2030    90   90
Polo                      35     2196   117  117
Puffer                    35     1946   110  110
Pullover                  35     2184   114  114
Relaxed Leg               35     2002    95   95
Shearling                 35     2169    95   95
Slim-Fit                  35     2155   119  119
Tracksuit Bottoms         35     2038    91   91
Trench Coat               35     2132   112  112
Windbreaker               35     2085   109  109
Wool                      35     1975   111  111

These are all the details of each product and product type.

In [53]:
sales_product_df = pd.merge(
    left=sales_df,
    right=product_df,
    how="left",
    left_on="product_id",
    right_on="product_id",
    suffixes=["_sal", "_prod"]
)
sales_product_df.head()

,sales_id,order_id,product_id,price_per_unit,quantity_sal,total_price,product_type,product_name,size,colour,price,quantity_prod,description
0,0,1,218,106,2,212,Shirt,Chambray,L,orange,105,44,"A orange coloured, L sized, Chambray Shirt"
1,1,1,481,118,1,118,Jacket,Puffer,S,indigo,110,62,"A indigo coloured, S sized, Puffer Jacket"
2,2,1,2,96,3,288,Shirt,Oxford Cloth,M,red,114,54,"A red coloured, M sized, Oxford Cloth Shirt"
3,3,1,1002,106,2,212,Trousers,Wool,M,blue,111,52,"A blue coloured, M sized, Wool Trousers"
4,4,1,691,113,3,339,Jacket,Parka,S,indigo,119,53,"A indigo coloured, S sized, Parka Jacket"


The results of the merge process above show a discrepancy between the values of price_per_unit and price. This may occur due to discounts, operational costs, and other expenses.

In [55]:
sales_product_df.groupby("product_type").agg({
    "sales_id": "nunique",
    "quantity_sal": "sum",
    "total_price": "sum"
})

,sales_id,quantity_sal,total_price
product_type,,,
Jacket,1676,3343,357026
Shirt,1641,3259,333600
Trousers,1683,3360,341174


Trousers are the best-selling product type. However, when considering the revenue generated, jackets are the product type that contributes the most to the company's revenue.

In [58]:
sales_product_df.groupby("product_name").agg({
    "sales_id": "nunique",
    "quantity_sal": "sum",
    "total_price": "sum"
}).sort_values("total_price", ascending=False)

,sales_id,quantity_sal,total_price
product_name,,,
Denim,273,527,52399
Casual Slim Fit,154,306,36414
Trench Coat,146,299,35581
Shearling,150,302,35334
Puffer,140,298,35164
Flannel,141,281,33158
Cropped,135,284,32660
Pleated,147,308,32340
Joggers,164,334,31062


Based on the pivot table, it is evident that denim products are the best sellers and also contribute the most revenue to the company.

# **Explore All Data**

In [59]:
all_df = pd.merge(
    left=sales_product_df,
    right=orders_customers_df,
    how="left",
    left_on="order_id",
    right_on="order_id"
)
all_df.head()

,sales_id,order_id,product_id,price_per_unit,quantity_sal,total_price,product_type,product_name,size,colour,...,customer_name,gender,age,home_address,zip_code,city,state,country,status,age_group
0,0,1,218,106,2,212,Shirt,Chambray,L,orange,...,fulan 64,Prefer not to say,75,4927 Alice MeadowApt. 960,7787,Sanfordborough,South Australia,Australia,Active,Seniors
1,1,1,481,118,1,118,Jacket,Puffer,S,indigo,...,fulan 64,Prefer not to say,75,4927 Alice MeadowApt. 960,7787,Sanfordborough,South Australia,Australia,Active,Seniors
2,2,1,2,96,3,288,Shirt,Oxford Cloth,M,red,...,fulan 64,Prefer not to say,75,4927 Alice MeadowApt. 960,7787,Sanfordborough,South Australia,Australia,Active,Seniors
3,3,1,1002,106,2,212,Trousers,Wool,M,blue,...,fulan 64,Prefer not to say,75,4927 Alice MeadowApt. 960,7787,Sanfordborough,South Australia,Australia,Active,Seniors
4,4,1,691,113,3,339,Jacket,Parka,S,indigo,...,fulan 64,Prefer not to say,75,4927 Alice MeadowApt. 960,7787,Sanfordborough,South Australia,Australia,Active,Seniors


In [63]:
all_df.groupby(["state", "product_type"]).agg({
    "quantity_sal": "sum",
    "total_price": "sum"
}).sort_values(["state", "total_price"], ascending=[True, False])

quantity_sal  total_price
state                        product_type                           
Australian Capital Territory Trousers               454        46790
                             Jacket                 406        43204
                             Shirt                  396        40448
New South Wales              Jacket                 451        47998
                             Shirt                  431        43980
                             Trousers               392        39766
Northern Territory           Trousers               384        38998
                             Jacket                 365        38991
                             Shirt                  336        33865
Queensland                   Jacket                 499        53511
                             Trousers               453        46045
                             Shirt                  417        42506
South Australia              Shirt                  509        52685
                             Jacket                 461        49090
                             Trousers               455        46041
Tasmania                     Jacket                 412        44370
                             Shirt                  387        39668
                             Trousers               379        38057
Victoria                     Trousers               414        41948
                             Shirt                  390        40206
                             Jacket                 359        38203
Western Australia            Trousers               429        43529
                             Jacket                 390        41659
                             Shirt                  393        40242

The pivot table above provides us with insights into the types of products favored by users based on their state locations. Notably, jackets are the most sold product type in the states of Queensland, Tasmania, and New South Wales.

In [68]:
all_df.groupby(["gender", "product_type"]).agg({
    "quantity_sal": "sum",
    "total_price": "sum"
}).sort_values(["gender", "total_price"], ascending=[True, False])

quantity_sal  total_price
gender            product_type                           
Female            Jacket                 481        50963
                  Trousers               481        49295
                  Shirt                  445        45146
Male              Jacket                 480        50978
                  Trousers               453        46071
                  Shirt                  410        41939
Prefer not to say Jacket                2382       255085
                  Shirt                 2404       246515
                  Trousers              2426       245808

It turns out that jackets are the most sold product type for all genders.

In [69]:
all_df.groupby(["age_group", "product_type"]).agg({
    "quantity_sal": "sum",
    "total_price": "sum"
}).sort_values(["age_group", "total_price"], ascending=[True, False])

quantity_sal  total_price
age_group product_type                           
Adults    Jacket                2292       245055
          Trousers              2272       231271
          Shirt                 2225       227781
Seniors   Jacket                 777        82959
          Trousers               766        77471
          Shirt                  733        74977
Youth     Trousers               322        32432
          Shirt                  301        30842
          Jacket                 274        29012

Here we can see that the jacket product type has succeeded in dominating the adults and seniors age groups, while the youth age group mostly buys trousers.